In [1]:
import torch
from torch.backends import cudnn
from face_detection import RetinaFace

import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from torch.utils.data import DataLoader 
import torchvision
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm import tqdm

from networks.sixDRepNet_networks import SixDRepNet
from utils import sixdrepnet_util
from utils.util import torch_fix_seed, get_video_name_list
from dataset import HPImageList
from preprocess import sixDRepNet_ImageTransform

In [2]:
torch_fix_seed()

In [3]:
hp_dataset = HPImageList(
    labels_path="/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/labels/PIMD_A/emo_and_au(video1-25).csv",
    video_name_list=get_video_name_list('/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/labels/PIMD_A/emo_and_au(video1-25)-video_name_list.csv'),
    hp_transform=sixDRepNet_ImageTransform(phase='test')
)

hp_dataloader = DataLoader(hp_dataset, batch_size=1, shuffle=False, num_workers=2)

In [4]:
device = (torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu"))
print(f"use device:{device}")

use device:cuda:0


In [5]:
#* 6DRepNet (Head Pose Estimatior)
hp_feat_extractor = SixDRepNet(backbone_name='RepVGG-B1g2', backbone_file='', deploy=True, pretrained=False)
hp_feat_extractor.load_state_dict(torch.load("/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/params/6DRepNet-snapshots/6DRepNet_300W_LP_AFLW2000.pth", map_location=device))

<All keys matched successfully>

In [6]:
for params in hp_feat_extractor.parameters():
    params.requires_grad = False
    
hp_feat_extractor.to(device)

hp_feat_extractor.eval()

SixDRepNet(
  (layer0): RepVGGBlock(
    (nonlinearity): ReLU()
    (se): Identity()
    (rbr_reparam): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  )
  (layer1): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (se): Identity()
      (rbr_reparam): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (1): RepVGGBlock(
      (nonlinearity): ReLU()
      (se): Identity()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
    )
    (2): RepVGGBlock(
      (nonlinearity): ReLU()
      (se): Identity()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): RepVGGBlock(
      (nonlinearity): ReLU()
      (se): Identity()
      (rbr_reparam): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=2)
    )
  )
  (layer2): Sequential(
    (0): RepVGGBlock(
      (nonlinearity): ReLU()
      (se): Identity()
      

In [7]:
pbar = tqdm(total=len(hp_dataloader))

hp_middle_features = []
hp_logits = []
r_matrixs = []
hp_list = []
img_path_list = []
with torch.no_grad():
    for i, batch in enumerate(hp_dataloader):
        imgs, img_paths, _ = batch
        imgs = imgs.to(device)
        
        # culc forward
        R_pred, hp_feats, logits = hp_feat_extractor(imgs)
        hp_middle_features += hp_feats.detach().cpu().numpy().tolist()
        hp_logits += logits.detach().cpu().numpy().tolist()
        r_matrixs += R_pred.T.reshape(1, -1).detach().cpu().numpy().tolist()
        
        euler = sixdrepnet_util.compute_euler_angles_from_rotation_matrices(R_pred)*180/np.pi
        p_pred_deg = euler[:, 0].cpu().detach().numpy()
        y_pred_deg = euler[:, 1].cpu().detach().numpy()
        r_pred_deg = euler[:, 2].cpu().detach().numpy()
        
        # save outputs
        img_path_list += img_paths
        hp_list.append([p_pred_deg[0], r_pred_deg[0], y_pred_deg[0]])
        
        # update tqdm bar
        pbar.update(1)
    
    path = '/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/interim/black.jpg'
    black_img = cv2.imread(path)
    black_img = sixDRepNet_ImageTransform(phase='test')(black_img)
    black_img = black_img.to(device)
    R_pred, hp_feats, logits = hp_feat_extractor(black_img.unsqueeze(0))
    hp_middle_features += hp_feats.detach().cpu().numpy().tolist()
    hp_logits += logits.detach().cpu().numpy().tolist()
    r_matrixs += R_pred.T.reshape(1, -1).detach().cpu().numpy().tolist()
    hp_list.append([0, 0, 0])
    img_path_list.append(path)
       
# close tqdm bar
pbar.close()

  0%|          | 0/19931 [00:00<?, ?it/s]

/tmp/ipykernel_1403842/4270110080.py:17: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1678402411778/work/aten/src/ATen/native/TensorShape.cpp:3571.)
  r_matrixs += R_pred.T.reshape(1, -1).detach().cpu().numpy().tolist()
100%|██████████| 19931/19931 [07:02<00:00, 47.18it/s]


In [8]:
df_path = pd.DataFrame(img_path_list, columns=["img_path"])
df_mid_hp = pd.DataFrame(hp_middle_features, columns=[i for i in range(2048)])
df_logits_hp = pd.DataFrame(hp_logits, columns=[i for i in range(6)])
df_r_matrixs = pd.DataFrame(r_matrixs, columns=[i for i in range(9)])
df_hp = pd.DataFrame(hp_list, columns=["pitch", "roll", "yaw"])

hp_list = pd.concat([df_path, df_hp], axis=1)
feat_list = pd.concat([df_path, df_mid_hp], axis=1)
logits_list = pd.concat([df_path, df_logits_hp], axis=1)
r_matrixs_list = pd.concat([df_path, df_r_matrixs], axis=1)

In [9]:
# hp_list.to_csv("/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/processed/PIMD_A/headpose.csv")
# feat_list.to_pickle("/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/processed/PIMD_A/6DRepNet_feature.pkl")
# logits_list.to_pickle("/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/processed/PIMD_A/6DRepNet_logits.pkl")
r_matrixs_list.to_pickle("/mnt/iot-qnap3/mochida/medical-care/emotionestimation/data/processed/PIMD_A/6DRepNet_r_matrixs.pkl")